# 侵入检测

检测目标位置是否被占用或侵入，常用于安防、货位检测


常用的侵入检测依据有颜色、线条、轮廓、角点等，各有一定的局限性。实际应用中根据现场环境使用多种方法综合判定。

在复杂环境或准确率要求高的场景，建议使用3D相机。

In [ ]:
from icamera.GigEHuaray import GigEHuaray
import cv2 as cv
# from threading import Thread
from queue import Empty
from multiprocessing import Process, Queue #, Pool
import os, time
from utils import show
import numpy as np

In [ ]:
GigEHuaray.enumCameras()

print(GigEHuaray.cameras)
camera = GigEHuaray('camera01')
camera.start()

In [ ]:
rawsites = list()
rawmarks = list()
rawareas = list()

def detect(frame):
    rawmarks.clear()
    rawsites.clear()
    
    ret, thresh1 = cv.threshold(frame, 60, 255 ,cv.THRESH_BINARY_INV) # thread参数可分析直方图得到
    # edge = cv.Canny(thresh1, 50, 400)
    # lines = cv.HoughLinesP(edge, 1, np.pi/180, 140, minLineLength=100) # rho(ρ)长度分辨率，角度分辨率
    # print(len(lines))
    # bgr=cv.cvtColor(frame, cv.COLOR_GRAY2BGR)
    # for line in lines.squeeze():
    #     x1, y1, x2, y2 = line
    #     cv.line(bgr, (x1, y1), (x2, y2), (0,0,255), 1)

    cnts, hiers = cv.findContours(thresh1, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    bgr = cv.merge([frame, frame, frame])
    for c in cnts:
        # print(cv.contourArea(c))
        area = cv.contourArea(c)
        if  area> 10000:
            x,y,w,h = cv.boundingRect(c)
            print(cv.boundingRect(c))
            cv.drawContours(bgr, [c], 0, (0,0,255), 3)
            cv.rectangle(bgr, (x, y), (x + w, y + h), (0,0,255), 3)
            rawmarks.append((x,y,w,h))
            rawsites.append(frame[y:y +h, x:x+w].copy())
            rawareas.append(area)
            # show(frame[y:y +h, x:x+w])
    print('show base sample...')
    show(bgr)


In [ ]:
'''
https://blog.csdn.net/xian0710830114/article/details/118786385
'''

import hashlib
import math
from skimage.metrics import structural_similarity


'''
    粗暴的md5比较 返回是否完全相同
'''
def md5_similarity(img1_path, img2_path):
    file1 = open(img1_path, "rb")
    file2 = open(img2_path, "rb")
    md = hashlib.md5()
    md.update(file1.read())
    res1 = md.hexdigest()
    md = hashlib.md5()
    md.update(file2.read())
    res2 = md.hexdigest()
    return res1 == res2
 
def normalize(data):
    return data / np.sum(data)
 
'''
    直方图相似度
    相关性比较 cv.HISTCMP_CORREL：值越大，相似度越高
    相交性比较 cv.HISTCMP_INTERSECT：值越大，相似度越高
    卡方比较 cv.HISTCMP_CHISQR：值越小，相似度越高
    巴氏距离比较 cv.HISTCMP_BHATTACHARYYA：值越小，相似度越高
'''
def hist_similarity(img1, img2, hist_size=256):
    imghistb1 = cv.calcHist([img1], [0], None, [hist_size], [0, 256])
    # imghistg1 = cv.calcHist([img1], [1], None, [hist_size], [0, 256])
    # imghistr1 = cv.calcHist([img1], [2], None, [hist_size], [0, 256])
 
    imghistb2 = cv.calcHist([img2], [0], None, [hist_size], [0, 256])
    # imghistg2 = cv.calcHist([img2], [1], None, [hist_size], [0, 256])
    # imghistr2 = cv.calcHist([img2], [2], None, [hist_size], [0, 256])
 
    distanceb = cv.compareHist(normalize(imghistb1), normalize(imghistb2), cv.HISTCMP_CORREL)
    # distanceg = cv.compareHist(normalize(imghistg1), normalize(imghistg2), cv.HISTCMP_CORREL)
    # distancer = cv.compareHist(normalize(imghistr1), normalize(imghistr2), cv.HISTCMP_CORREL)
    # meandistance = np.mean([distanceb, distanceg, distancer])
    meandistance = distanceb
    return meandistance
 
def PSNR(img1, img2):
    mse = np.mean((img1/255. - img2/255.) ** 2)
    if mse == 0:
        return 100
    PIXEL_MAX = 1
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))
 
def SSIM(img1, img2):
    gray1 = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)
    gray2 = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)
    # 计算两个灰度图像之间的结构相似度
    score, diff = structural_similarity(gray1, gray2, win_size=101, full=True)
    # diff = (diff * 255).astype("uint8")
    # print("SSIM:{}".format(score))
    return score, diff


In [ ]:
%%time

def compare(frame):
    bgr = cv.merge([frame, frame, frame])
    threshold = 0.95 # 差异度阈值   

    for i in range(0, len(rawmarks)):
        x, y , w, h = rawmarks[i]
        # site = cv.resize(rawsites[i], (25, 20))
        # s = cv.resize(frame[y:y +h, x:x+w], (25, 20))
        s = frame[y:y +h, x:x+w]
        ret, s = cv.threshold(s, 60, 255 ,cv.THRESH_BINARY_INV)
        # kernel = cv.getStructuringElement(cv.MORPH_RECT, (5,5))
        # img_close = cv.morphologyEx(s, cv.MORPH_CLOSE, kernel) 
        # show(img_close)

        # # diff = PSNR(site, s)
        # # diff = hist_similarity(site, s)
        # lines = cv.HoughLinesP(img_close, 3, np.pi/180, 40, minLineLength=200, maxLineGap=1) 
        # for line in lines.squeeze():
        #     x1, y1, x2, y2 = line
        #     cv.line(s, (x1, y1), (x2, y2), (0,0,255), 1)
        
        cnts, hiers = cv.findContours(s, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        diff=0
        for c in cnts:
            # print(cv.contourArea(c))
            area = cv.contourArea(c)
            if  area> 10000:
                diff = area/ rawareas[i]
                break
                
        color = (0, 255, 0) if diff > threshold else (0,0,255)
        cv.rectangle(bgr, (x, y), (x + w, y + h), color, 3)
        
    return cv.resize(bgr, (900, 800))

# frame = cv.imread('intrusion_detection.bmp', 0)
# detect(frame)
# show(draw(frame))

In [ ]:

while not camera.imgFactory.empty(): # clear buffers
    camera.imgFactory.get()
    
while(1):
    try:
        frame = camera.imgFactory.get(block=False) 
        x, y, w, h = (320, 168, 1024, 920)# Width=1024, Height=920, OffsetX=320,OffsetY=168
        frame = frame[y:y+h, x:x+w]
        # cv.imwrite('intrusion_detection.bmp', frame)
    except Empty:
        continue

    if len(rawsites)==0:
        detect(frame)
        
    s = time.time()
    img = compare(frame)
    # print(f"compare times: {time.time()-s:.3f}")
    
    cv.imshow(camera.name, img)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break
    
camera.close()
cv.destroyAllWindows()